### Scripts used to run different metagenome binning tools

In [ ]:
#!/bin/bash
#SBATCH -J "binning"
#SBATCH -p gpu
#SBATCH -A scc\_users
#SBATCH -t 01-00:00:00
#SBATCH -N 1
#SBATCH -n 1
#SBATCH -c 24
#SBATCH -G 2
#SBATCH --mem=128G

DATAPATH="/scratch1/users/yazhini.a01/binning_benchmarking/strain_madness/megahit_assembly/pooled_assembly"
source ~/miniforge3/bin/activate
"/usr/bin/time -v vamb --outdir ${DATAPATH}/vamb\_results --fasta ${DATAPATH}/single\_pooled\_final.contigs.fa --bamfiles ${DATAPATH}/bamfiles/*.bam -m 1000 --minfasta 200000 --cuda -p 24"

conda activate metabat2\_env
"/usr/bin/time -v metabat2 -i ${DATAPATH}/sorted\_pooled\_final.contigs.fa -a ${DATAPATH}/abundances\_gf\_sorted.tsv -o ${DATAPATH}/metabat2\_results/metabat2\_results -t 24 -m 1500"
conda deactivate


conda activate mcdevol\_env
"/usr/bin/time -v /scratch1/users/yazhini.a01/binning\_benchmarking/binning\_tool/mcdevol/mcdevol.py -c ${DATAPATH}/sorted\_pooled\_final.contigs.fa -o ${DATAPATH}/mcdevol\_results -a ${DATAPATH}/abundances\_gf\_sorted.tsv --abundformat metabat -n 24"
conda deactivate

conda activate genomeface\_env
"/usr/bin/time -v genomeface -i ${DATAPATH}/sorted\_pooled\_final.contigs.fa -o ${DATAPATH}/genomeface\_results -a ${DATAPATH}/abundances\_gf\_sorted.tsv -g marker\_hits -m 1000"
conda deactivate


conda activate metawrap-env
"/usr/bin/time -v metawrap binning -o ${DATAPATH}/metawrap -t 24 -a ${DATAPATH}/sorted\_pooled\_final.contigs.fa --metabat2 --maxbin2 --concoct --interleaved /cbscratch/yazhini/binning\_benchmarking/strain\_madness/fastqfiles/coco\_corrected/*.fastq"
"/usr/bin/time -v metawrap bin\_refinement -o ${DATAPATH}/metawrap/bin\_refined -t 24 -A ${DATAPATH}/metawrap/metabat2\_bins/ -B ${DATAPATH}/metawrap/maxbin2\_bins/ -C ${DATAPATH}/metawrap/concoct\_bins/ -c 50 -x 10"
conda deactivate


module load anaconda3/2021.05
conda activate comebin\_env
"/usr/bin/time -v /scratch1/users/yazhini.a01/binning\_benchmarking/COMEBin/bin/run\_comebin.sh -a ${DATAPATH}/single\_pooled\_final.contigs.fa -o ${DATAPATH}/comebin\_results -p ${DATAPATH}/bamfiles -t 24"
conda deactivate




